In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
#os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

True

In [3]:
import os
#os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [13]:
# from langchain_huggingface import HuggingFaceEmbeddings
# embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [89]:
len(embeddings.embed_query("hello AI"))

768

| Feature                  | SQL/NoSQL       | Vector DB   |
| ------------------------ | --------------- | ----------- |
| Exact match search       | Supported     |    Supported     |
| Similarity search        | Not supported | Optimized |
| High-dim vector support  | Inefficient   | Native    |
| ANN Indexing (IVF, HNSW) |Not present   | Present   |
| Use-case fit for RAG     | No            |    Yes       |


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
documents=["what is a capital of USA",
           "who is a president of usa",
           "who is a prime minister of india"]

my_query="narendra modi is indian prime minister"

In [18]:
query_embedding=embeddings.embed_query(my_query)
document_embedding=embeddings.embed_documents(documents)

In [19]:
scores=cosine_similarity([query_embedding],document_embedding)

In [20]:
scores

array([[0.44249935, 0.55830709, 0.68122836]])

In [49]:
from sklearn.metrics.pairwise import euclidean_distances

documents = [
    "what is a capital of USA",
    "who is a president of usa",
    "who is a prime minister of india"
]

my_query = "narendra modi is indian prime minister"
query_embedding = embeddings.embed_query(my_query)
document_embedding = embeddings.embed_documents(documents)

# L2 distances
distances = euclidean_distances([query_embedding], document_embedding)

# Lower distance = more similar, so convert to similarity (optional)
similarity_scores = distances  # invert to use as "similarity"

In [50]:
similarity_scores

array([[1.0559354 , 0.93988533, 0.79846248]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on **direction** (angle) only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [21]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [24]:
len(embeddings.embed_query("hello world"))

768

In [25]:
index = faiss.IndexFlatL2(768)

In [23]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000192C47CAD30> >

In [27]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [33]:
vector_store.add_texts(["AI is powerful", "Dogs are cute","AI is future"])

['502de898-663a-4b01-bbc6-c35b7f220229',
 'a43d30d2-11bc-44c0-93ba-b7843e7c1f9e',
 '4c648622-fa97-49d0-b4f7-8a8cec7f5512']

In [38]:
vector_store.index_to_docstore_id

{0: '392c7677-7298-4699-a781-70cd65a78a05',
 1: '4d010d1c-0108-4bfa-8517-2639527b9362',
 2: '502de898-663a-4b01-bbc6-c35b7f220229',
 3: 'a43d30d2-11bc-44c0-93ba-b7843e7c1f9e',
 4: '4c648622-fa97-49d0-b4f7-8a8cec7f5512'}

In [39]:
faiss_index_id = 1
docstore_id = vector_store.index_to_docstore_id[faiss_index_id]
doc_text = vector_store.docstore.search(docstore_id).page_content

In [40]:
doc_text

'Dogs are cute'

In [36]:
results = vector_store.similarity_search("Tell me about AI", k=2)

In [37]:
results

[Document(id='392c7677-7298-4699-a781-70cd65a78a05', metadata={}, page_content='AI is powerful'),
 Document(id='502de898-663a-4b01-bbc6-c35b7f220229', metadata={}, page_content='AI is powerful')]

In [41]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents)

['626914b3-e077-4881-b440-23a999f1c1fc',
 '7b53c746-fb3d-4eb5-934d-36867789c901',
 '968f0e78-d6ee-4d53-a1f2-39b5e15bb5cf',
 '8bd196b1-52e7-4062-a18f-8de1a0ac264c',
 '4b005b06-1537-4004-a060-9fb7ad042bd0',
 '2728bf2e-6fc0-4386-bb77-30d7e10e335a',
 '458b2816-bb60-4319-8c64-2239c2dedbdc',
 '900e97a1-999e-4197-b2c4-255e4777ff81',
 'd5bfa0ec-c535-4c46-8993-8f53bd73125d',
 'fee0f5bb-61b1-43e8-b1fd-015332381d82']

In [42]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [43]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": {"$eq": "tweet"}},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [44]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.664568] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [45]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='8bd196b1-52e7-4062-a18f-8de1a0ac264c', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [46]:
vector_store.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search("qux")

In [51]:
import faiss
import numpy as np
faiss_index = faiss.IndexFlatIP(768)

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [52]:
from langchain_community.document_loaders import PyPDFLoader

In [53]:
file_path = (
    "C:\Complete_Content2\Agentic-2.0\data\llama2.pdf"
)

In [ ]:
file_path = (
    "C:\Complete_Content2\Agentic-2.0\data\llama2.pdf"
)
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [56]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [57]:
pages

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Complete_Content2\\Agentic-2.0\\data\\llama2.pdf', 'total_pages': 77, 'page': 0, 'page_label': '1'}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit

In [61]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [62]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# 5. Apply splitter
split_docs = splitter.split_documents(pages)

In [63]:
split_docs

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Complete_Content2\\Agentic-2.0\\data\\llama2.pdf', 'total_pages': 77, 'page': 0, 'page_label': '1'}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev'),
 Do

In [64]:
vector_store.add_documents(documents=split_docs)

['b09be386-bc82-438d-95d8-e6724ed6641f',
 '20e4c864-0f7b-4823-aa27-037a4b3f61a9',
 'ed83b305-4f90-4144-8e96-a1c6253ca526',
 '563a389a-38ab-46c3-aede-082728577a6e',
 'b8f37d75-83e0-4df1-bb56-4550820a21a7',
 '53bb2c80-a6d4-41f4-a289-e7cd79ffa88f',
 'ed08b4d5-6d79-41fb-933f-2946cff5837f',
 '68a4ff22-7357-41d7-a37b-b8cec62f4e78',
 '4f78a5cf-52c8-4a15-82f8-ab5cab76cd9d',
 '22d50087-550c-4f8b-b30b-88b44872e61e',
 '586d8a39-f0df-4778-b44b-c6a53d8ed7e1',
 'e5e7c9ed-063d-4aa1-b2f8-52b380df0d3a',
 '556712d4-38ac-48eb-82fb-6e8613c035b0',
 'f28c980c-a4a5-4197-b209-032457c84c74',
 'fc629064-4f06-4098-8a45-d0a1736003f5',
 '76875f6a-f0ba-4231-81b0-71b3335b8441',
 'bbe9d288-a7a4-476a-8853-b9b8112f4cd3',
 'd0a8c929-c075-43d1-a53e-cd3d93268f13',
 '187ab2aa-6eb7-4a67-9dda-794e6632b30d',
 '58335b86-c37a-41e6-ae5d-33dc6e4dd057',
 '88c07016-0b82-4db3-ba4b-6408f3285955',
 '342a1eda-f622-4cb0-9a8a-829d240a1e65',
 '1babb120-743c-49ee-abfd-3a1311caf132',
 'cf31bf7f-9ef1-4770-b507-44f099a5ee1f',
 '9419f520-ea3e-

In [66]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("what is llama2?")

[Document(id='d2282a0e-0d13-4835-85e4-b625a9f4a433', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Complete_Content2\\Agentic-2.0\\data\\llama2.pdf', 'total_pages': 77, 'page': 76, 'page_label': '77'}, page_content='Llama 2is a new technology that carries risks with use. Testing conducted to date has been in\nEnglish, and has not covered, nor could it cover all scenarios. For these reasons, as with all LLMs,\nLlama 2’s potential outputs cannot be predicted in advance, and the model may in some instances\nproduce inaccurate or objectionable responses to user prompts. Therefore, before deploying any\napplications ofLlama 2, developers should perform safety testing and tuning ta

In [68]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Complete_Content2\Agentic-2.0\env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [69]:
import pprint

In [70]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [71]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [72]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [73]:
from langchain_core.runnables import RunnablePassthrough

In [84]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [85]:
# Chain
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [86]:
rag_chain.invoke("What is Task Decomposition?")

'I am sorry, but this document does not contain the answer to what task decomposition is.  I do not know.'